## Processamento de Linguagem Natural
### Atividade 01
#### Aluno: Dayvison Gomes de Oliveira
#### Matrícula: 20180036876


In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.naive_bayes import MultinomialNB

In [2]:
df = pd.read_csv('B2W-Reviews01.csv')

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
df.head()

,submission_date,reviewer_id,product_id,product_name,product_brand,site_category_lv1,site_category_lv2,review_title,overall_rating,recommend_to_a_friend,review_text,reviewer_birth_year,reviewer_gender,reviewer_state
0,2018-01-01 00:11:28,d0fb1ca69422530334178f5c8624aa7a99da47907c44de...,132532965,Notebook Asus Vivobook Max X541NA-GO472T Intel...,NaN,Informática,Notebook,Bom,4,Yes,Estou contente com a compra entrega rápida o ú...,1958.0,F,RJ
1,2018-01-01 00:13:48,014d6dc5a10aed1ff1e6f349fb2b059a2d3de511c7538a...,22562178,Copo Acrílico Com Canudo 500ml Rocie,NaN,Utilidades Domésticas,"Copos, Taças e Canecas","Preço imbatível, ótima qualidade",4,Yes,"Por apenas R$1994.20,eu consegui comprar esse ...",1996.0,M,SC
2,2018-01-01 00:26:02,44f2c8edd93471926fff601274b8b2b5c4824e386ae4f2...,113022329,Panela de Pressão Elétrica Philips Walita Dail...,philips walita,Eletroportáteis,Panela Elétrica,ATENDE TODAS AS EXPECTATIVA.,4,Yes,SUPERA EM AGILIDADE E PRATICIDADE OUTRAS PANEL...,1984.0,M,SP
3,2018-01-01 00:35:54,ce741665c1764ab2d77539e18d0e4f66dde6213c9f0863...,113851581,Betoneira Columbus - Roma Brinquedos,roma jensen,Brinquedos,Veículos de Brinquedo,presente mais que desejado,4,Yes,MEU FILHO AMOU! PARECE DE VERDADE COM TANTOS D...,1985.0,F,SP
4,2018-01-01 01:00:28,7d7b6b18dda804a897359276cef0ca252f9932bf4b5c8e...,131788803,"Smart TV LED 43"" LG 43UJ6525 Ultra HD 4K com C...",lg,TV e Home Theater,TV,"Sem duvidas, excelente",5,Yes,"A entrega foi no prazo, as americanas estão de...",1994.0,M,MG


In [4]:
df = df.dropna(subset=['review_text']).reset_index().drop(columns=['index'])

In [5]:
df['overall_rating'] = np.where(df['overall_rating'] >= 3, 'pos', 'neg')

Classificador Naive Bayes antes do tratamento "NAO_".

Resolvi vetorizar antes de separar.

In [8]:
vect = TfidfVectorizer()
text_vect = vect.fit_transform(df['review_text'])

X_train, X_test, y_train, y_test = train_test_split(text_vect, df['overall_rating'], test_size=0.2)

In [9]:
clf = MultinomialNB()
clf.fit(X_train, y_train)

print(classification_report(y_test, clf.predict(X_test)))

              precision    recall  f1-score   support

         neg       0.86      0.70      0.77      6697
         pos       0.90      0.96      0.93     19123

    accuracy                           0.89     25820
   macro avg       0.88      0.83      0.85     25820
weighted avg       0.89      0.89      0.89     25820



Após o tratamento "NAO_"

In [10]:
negative_tokens = ['não','nao','jamais','nem','nunca','tampouco']
pont = [',','.','!',';','?']

In [12]:
def func(text):
    frase = text.lower().split()
    
    for i in range(len(frase)):
        
        if frase[i] in negative_tokens:
            
            for j in range(i,len(frase) -1):
                
                if frase[j][-1] in pont:
                    break
                    
                else:
                    frase[j+1] = 'NAO_' + frase[j+1]

    return " ".join(frase)


In [13]:
text = df['review_text'].apply(func)

In [24]:
func('eu nao gostei do filme, foi uma merda.')

'eu nao NAO_gostei NAO_do NAO_filme, foi uma merda.'

In [15]:
so_neg = text[ np.where( df['overall_rating'] == 'neg', True, False  )  ]

In [16]:
so_neg.iloc[0]

'meu produto nao NAO_foi NAO_entregue NAO_e NAO_a NAO_americanas NAO_esta NAO_descontando NAO_na NAO_fatura NAO_do NAO_meu NAO_cartão'

In [17]:
so_neg.iloc[5]

'não NAO_recebi NAO_segundo NAO_o NAO_correio NAO_o NAO_endereço NAO_estava NAO_errado! porém o outro pedido recebi!'

In [25]:
vect = TfidfVectorizer()
text_vect = vect.fit_transform(text)

In [26]:
X_train, X_test, y_train, y_test = train_test_split(text_vect, df['overall_rating'], test_size=0.2)

In [27]:
clf = MultinomialNB()
clf.fit(X_train, y_train)

MultinomialNB()

In [28]:
print(classification_report(y_test, clf.predict(X_test)))

              precision    recall  f1-score   support

         neg       0.84      0.74      0.79      6845
         pos       0.91      0.95      0.93     18975

    accuracy                           0.89     25820
   macro avg       0.87      0.84      0.86     25820
weighted avg       0.89      0.89      0.89     25820

